In [3]:
import cv2
import numpy as np
import math

In [4]:
class ImageHomography:
    '''
        利用sift特征求两张图片的单应性变换
    '''
    def __init__(self, root_path):
        self.root_path = root_path
        self.query_path = root_path + "left.jpg"
        self.train_path = root_path + 'right.jpg'
        self.query_img = cv2.imread(self.query_path, 0)
        self.train_img = cv2.imread(self.train_path, 0)
        self.sift = cv2.SIFT_create()

    def detect_and_compute(self):
        '''
            检测关键点并计算描述符
        '''
        self.kp1, self.des1 = self.sift.detectAndCompute(self.query_img, None)
        self.kp2, self.des2 = self.sift.detectAndCompute(self.train_img, None)
    
    def match_features(self):
        '''
            使用FLANN匹配器进行特征匹配
        '''
        # 配置FLANN匹配器参数
        index_params = dict(algorithm=1, trees=5)
        search_params = dict(checks=50)
        flann = cv2.FlannBasedMatcher(index_params, search_params)\
        
        # 使用k-NN匹配描述符
        self.matches = flann.knnMatch(self.des1, self.des2, k=2)

        # 使用Lowe's比例测试来筛选匹配
        self.good_matches = []
        for m, n in self.matches:
            if m.distance < 0.75 * n.distance:
                self.good_matches.append(m)
    
    def find_homography(self):
        '''
            计算图像之间的单应性变换矩阵
        '''
        if len(self.good_matches) > 4:
            # 确保有足够的匹配点来计算单应性矩阵
            src_pts = np.float32([self.kp1[m.queryIdx].pt for m in self.good_matches]).reshape(-1, 1, 2)
            dst_pts = np.float32([self.kp2[m.trainIdx].pt for m in self.good_matches]).reshape(-1, 1, 2)
            
            # 使用RANSAC算法计算单应性矩阵
            self.H, self.mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)

    def draw_matches(self):
        '''
            绘制匹配结果并保存
        '''
        matchesMask = self.mask.ravel().tolist()
        draw_params = dict(matchColor=(0, 255, 0), singlePointColor=None, matchesMask=matchesMask, flags=2)
        result_img = cv2.drawMatches(self.query_img, self.kp1, self.train_img, self.kp2, self.good_matches, None, **draw_params)
        cv2.imwrite(self.root_path + 'sift_features_matching.jpg', result_img)

    def warp_image(self):
        """
            对原始图像进行透视变换并保存结果。
        """
        h, w = self.query_img.shape
        im_out = cv2.warpPerspective(cv2.imread(self.train_path), self.H, (w, h))
        cv2.imwrite(self.root_path + 'sift_result.jpg', im_out)

    def process(self):
        """
            处理图像：检测特征、匹配特征、计算单应性矩阵、绘制匹配结果、进行透视变换。
        """
        self.detect_and_compute()
        self.match_features()
        self.find_homography()
        self.draw_matches()
        self.warp_image()
        print(f'Processing completed, results saved in {self.root_path}')

In [5]:
if __name__ == '__main__':
    root_path = "src_img/"
    image_homography = ImageHomography(root_path)
    image_homography.process()

Processing completed, results saved in src_img/
